<a href="https://colab.research.google.com/github/khadijakaleem/Face_detection_through_landmark/blob/main/Copy_of_LandmarkCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [ ]:
import zipfile
import os
import dlib
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json

# Unzipping the file
zip_file_path = '/content/ImageDataset.zip'  # Path to the uploaded zip file
extracted_folder_path = '/content/ImageDataset'  # Destination folder

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print("Unzipping completed.")

# Install dlib
!pip install dlib

# Download pre-trained model for facial landmark detection
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

Unzipping completed.
--2024-08-02 07:13:04--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  46.2MB/s    in 1.3s    

2024-08-02 07:13:05 (46.2 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
# Initialize dlib's face detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Function to perform facial landmarking and save to JSON and annotated image
def detect_landmarks(image_path, output_json, output_image):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
    if image is None:
        print(f"Error loading image: {image_path}")
        return

    # Resize the image if it's too small
    height, width = image.shape
    if height < 96 or width < 96:
        image = cv2.resize(image, (96, 96))  # Resize to 96x96 (or other appropriate size)

    rects = detector(image, 1)

    landmarks = []
    for (i, rect) in enumerate(rects):
        shape = predictor(image, rect)
        shape = [(p.x, p.y) for p in shape.parts()]
        landmarks.append(shape)

    # Save landmarks to JSON
    with open(output_json, 'w') as json_file:
        json.dump(landmarks, json_file)

    # Annotate and save the image
    color_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)  # Convert grayscale to RGB for visualization
    for shape_set in landmarks:
        for (x, y) in shape_set:
            cv2.circle(color_image, (x, y), 1, (0, 255, 0), -1)

    cv2.imwrite(output_image, color_image)  # Save the annotated image

# Function to process images in a given emotion folder
def process_emotion_folder(emotion_folder_path, output_folder_path):
    # Ensure the output folder exists
    os.makedirs(output_folder_path, exist_ok=True)

    for image_file in os.listdir(emotion_folder_path):
        image_path = os.path.join(emotion_folder_path, image_file)
        if image_path.lower().endswith('.jpg'):
            output_json = os.path.join(output_folder_path, os.path.splitext(image_file)[0] + '.json')
            output_image = os.path.join(output_folder_path, image_file)
            print(f"Processing {image_path} and saving landmarks to {output_json} and {output_image}")
            detect_landmarks(image_path, output_json, output_image)

# Example usage:
# Specify the path to the specific emotion folder you want to process
emotion_folder_path = '/content/ImageDataset/ImageDataset/train/surprise'  # Change this to the path of your specific emotion folder
output_folder_path = '/content/ImageDataset/ImageDataset/train/annotated_surprise'  # Change this to the desired output folder

process_emotion_folder(emotion_folder_path, output_folder_path)


Processing /content/ImageDataset/ImageDataset/train/surprise/13447.jpg and saving landmarks to /content/ImageDataset/ImageDataset/train/annotated_surprise/13447.json and /content/ImageDataset/ImageDataset/train/annotated_surprise/13447.jpg
Processing /content/ImageDataset/ImageDataset/train/surprise/3888.jpg and saving landmarks to /content/ImageDataset/ImageDataset/train/annotated_surprise/3888.json and /content/ImageDataset/ImageDataset/train/annotated_surprise/3888.jpg
Processing /content/ImageDataset/ImageDataset/train/surprise/30112.jpg and saving landmarks to /content/ImageDataset/ImageDataset/train/annotated_surprise/30112.json and /content/ImageDataset/ImageDataset/train/annotated_surprise/30112.jpg
Processing /content/ImageDataset/ImageDataset/train/surprise/2616.jpg and saving landmarks to /content/ImageDataset/ImageDataset/train/annotated_surprise/2616.json and /content/ImageDataset/ImageDataset/train/annotated_surprise/2616.jpg
Processing /content/ImageDataset/ImageDataset/

In [ ]:
#save the folder annotated emotion in a zip format

import shutil

# Zip the annotated emotion folder
output_zip_path = '/content/annotated_angry.zip'
shutil.make_archive(output_zip_path[:-4], 'zip', '/content/ImageDataset/ImageDataset/train/annotated_angry')

print(f"Annotated images zipped to {output_zip_path}")


Annotated images zipped to /content/annotated_angry.zip


In [ ]:
#Deletes those jpg files which have not been annotated and saves those jpg files for which the corresponding json file actually has values
#i.e images that were detected

import os
import json

# Paths to the folders
surprise_folder = '/content/ImageDataset/ImageDataset/train/surprise'
annotated_surprise_folder = '/content/ImageDataset/ImageDataset/train/annotated_surprise'

# Iterate through JSON files in annotated_surprise folder
for filename in os.listdir(annotated_surprise_folder):
    if filename.endswith('.json'):
        json_path = os.path.join(annotated_surprise_folder, filename)

        # Load and check if JSON is empty
        with open(json_path, 'r') as f:
            landmarks = json.load(f)
        if not landmarks:  # Check if the list is empty
            print(f"Found empty JSON: {json_path}")

            # Delete the empty JSON file
            os.remove(json_path)

            # Construct corresponding image filenames
            image_filename = filename.replace('.json', '.jpg')
            surprise_image_path = os.path.join(surprise_folder, image_filename)
            annotated_image_path = os.path.join(annotated_surprise_folder, image_filename)

            # Delete corresponding images if they exist
            if os.path.exists(surprise_image_path):
                os.remove(surprise_image_path)
                print(f"Deleted corresponding image from surprise: {surprise_image_path}")
            if os.path.exists(annotated_image_path):
                os.remove(annotated_image_path)
                print(f"Deleted corresponding image from annotated_surprise: {annotated_image_path}")

print("Cleanup complete.")


Found empty JSON: /content/ImageDataset/ImageDataset/train/annotated_surprise/25353.json
Deleted corresponding image from surprise: /content/ImageDataset/ImageDataset/train/surprise/25353.jpg
Deleted corresponding image from annotated_surprise: /content/ImageDataset/ImageDataset/train/annotated_surprise/25353.jpg
Found empty JSON: /content/ImageDataset/ImageDataset/train/annotated_surprise/13331.json
Deleted corresponding image from surprise: /content/ImageDataset/ImageDataset/train/surprise/13331.jpg
Deleted corresponding image from annotated_surprise: /content/ImageDataset/ImageDataset/train/annotated_surprise/13331.jpg
Found empty JSON: /content/ImageDataset/ImageDataset/train/annotated_surprise/21758.json
Deleted corresponding image from surprise: /content/ImageDataset/ImageDataset/train/surprise/21758.jpg
Deleted corresponding image from annotated_surprise: /content/ImageDataset/ImageDataset/train/annotated_surprise/21758.jpg
Found empty JSON: /content/ImageDataset/ImageDataset/tr

In [ ]:
# to delete those json files in angry folder which do not have a same name .jpg file in angry itself.

# Path to the angry folder
disgust_folder = '/content/ImageDataset/ImageDataset/train/angry'

# Get list of all JPG files in angry folder
jpg_files = [f for f in os.listdir(disgust_folder) if f.endswith('.jpg')]

# Iterate through JSON files in angry folder
for filename in os.listdir(disgust_folder):
    if filename.endswith('.json'):
        corresponding_jpg = filename[:-5] + '.jpg'  # Check for corresponding JPG
        if corresponding_jpg not in jpg_files:
            json_path = os.path.join(disgust_folder, filename)
            os.remove(json_path)
            print(f"Deleted: {json_path} (no corresponding JPG)")


In [ ]:
# prompt: write code to zip annotated_digust folder

# Zip the annotated_disgust folder
output_zip_path = '/content/annotated_disgust.zip'
shutil.make_archive(output_zip_path[:-4], 'zip', '/content/ImageDataset/ImageDataset/train/annotated_disgust')

print(f"Annotated images zipped to {output_zip_path}")


Annotated images zipped to /content/annotated_disgust.zip


In [ ]:
#compare surprise and annotated_surprise folders. deletes the .jpg files from sad that are not present in annotated_surprise folder. copies the json files in annotated surprise folder and put them in surprise folder

# Paths to the folders
surprise_folder = '/content/ImageDataset/ImageDataset/train/surprise'
annotated_surprise_folder = '/content/ImageDataset/ImageDataset/train/annotated_surprise'

# Get lists of files in each folder
surprise_files = set(os.listdir(surprise_folder))
annotated_surprise_files = set(os.listdir(annotated_surprise_folder))

# Find JPG files in surprise that are not in annotated_surprise
files_to_delete = [f for f in surprise_files if f.endswith('.jpg') and f not in annotated_surprise_files]

# Delete those files from surprise
for file in files_to_delete:
    file_path = os.path.join(surprise_folder, file)
    os.remove(file_path)
    print(f"Deleted: {file_path}")

# Copy JSON files from annotated_surprise to surprise
for file in annotated_surprise_files:
    if file.endswith('.json'):
        src_path = os.path.join(annotated_surprise_folder, file)
        dst_path = os.path.join(surprise_folder, file)
        shutil.copy(src_path, dst_path)
        print(f"Copied: {src_path} to {dst_path}")

print("Comparison and cleanup complete.")


Copied: /content/ImageDataset/ImageDataset/train/annotated_surprise/24328.json to /content/ImageDataset/ImageDataset/train/surprise/24328.json
Copied: /content/ImageDataset/ImageDataset/train/annotated_surprise/25149.json to /content/ImageDataset/ImageDataset/train/surprise/25149.json
Copied: /content/ImageDataset/ImageDataset/train/annotated_surprise/6742.json to /content/ImageDataset/ImageDataset/train/surprise/6742.json
Copied: /content/ImageDataset/ImageDataset/train/annotated_surprise/30623.json to /content/ImageDataset/ImageDataset/train/surprise/30623.json
Copied: /content/ImageDataset/ImageDataset/train/annotated_surprise/27622.json to /content/ImageDataset/ImageDataset/train/surprise/27622.json
Copied: /content/ImageDataset/ImageDataset/train/annotated_surprise/29508.json to /content/ImageDataset/ImageDataset/train/surprise/29508.json
Copied: /content/ImageDataset/ImageDataset/train/annotated_surprise/11125.json to /content/ImageDataset/ImageDataset/train/surprise/11125.json
C

In [ ]:
#zip the sad file

# Zip the sad folder
output_zip_path = '/content/surprise.zip'
shutil.make_archive(output_zip_path[:-4], 'zip', '/content/ImageDataset/ImageDataset/train/surprise')

print(f"Surprise images zipped to {output_zip_path}")


Sad images zipped to /content/surprise.zip
